In [128]:
#Imports
from google.cloud import vision
import io
import pandas as pd
import string
import re
import requests

In [129]:
#Definiciones claves
string_punctuation = '!"#$%&r"\"r"'"()*+,./:;<=>?@[r"
string_punctuation2 =  "\\""]^_`{|}~"r"\n"

In [130]:
#Función de detección de descripción
def find_description(text):
    description_list = []
    for word in text:
        for i in range(len(data["names"])):
            if word == data["names"][i].lower():
                description_list.append(data["description"][i])
    print(f"Found {round(len(description_list)/len(text)*100,2)}% of ingredients")
    return description_list

In [131]:
data = pd.read_csv('incidecoder_descr_clean_sinpunc.csv', sep=",")
data = data.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)

data.head()

,names,links,description
0,Aluminum Starch Octenylsuccinate,https://incidecoder.com//ingredients/aluminum-...,A handy helper ingredient that comes in a whit...
1,Amaranthus Caudatus Seed Extract,https://incidecoder.com//ingredients/amaranthu...,Its the seed extract of a plant called amarant...
2,Aminomethyl Propanol,https://incidecoder.com//ingredients/aminometh...,An alkaline high pH aka basic material that is...
3,Aminopropyl Ascorbyl Phosphate,https://incidecoder.com//ingredients/aminoprop...,A vitamin C derivative thats created by combin...
4,Ammonium Acryloyldimethyltaurate/​VP Copolymer,https://incidecoder.com//ingredients/ammonium-...,A kind of polymer big molecule from repeated s...


In [132]:
# Listado ingredientes de la API   
url = 'https://skincare-api.herokuapp.com/products'
response = requests.get(url).json()
products= pd.DataFrame(response)
products = products[['name','ingredient_list']]

In [133]:
products

,name,ingredient_list
0,age spot brightening pen,"[water, butylene glycol, alcohol, dipropylene ..."
1,all day balancing care serum,"[camellia sinensis leaf water, phyllostachis b..."
2,bio-enzyme refining complex,"[panax ginseng root extract, cyclopentasiloxan..."
3,color control cushion compact spf50,"[phyllostachis bambusoides juice, cyclopentasi..."
4,future response age defense cr̬me,"[camellia sinensis leaf water, dicaprylyl carb..."
...,...,...
1735,Green Tea Mist,"[Water, Green tea]"
1736,Green Tea Mist,"[Water, Green tea]"
1737,Jojoba Oil,[Jojoba Oil]
1738,Mafuta,"[Huile de ricin, Aloe vera]"


# Tipear producto (Buscador)

In [214]:
#Función de detección de descripción
def find_product_description(product_name):
    product_description_list = []
    for i in range(len(products["name"])):
        if product_name == (products["name"][i].lower()):
            product_description_list.append(products["ingredient_list"][i])
    return product_description_list[0]

In [215]:
product_name = input("Insert product name ")

Insert product name all day balancing care serum


In [216]:
find_product_description(product_name)

['camellia sinensis leaf water',
 'phyllostachis bambusoides juice',
 'panax ginseng root extract',
 'butylene glycol',
 'alcohol',
 'carica papaya (papaya) fruit extract',
 'water',
 'betula platyphylla japonica juice',
 'betaine',
 'hibiscus abelmoschus seed extract',
 'dipotassium glycyrrhizate',
 'sodium hyaluronate',
 'beta-glucan',
 'natto gum',
 'tricholoma matsutake extract',
 'camellia sinensis callus culture conditioned media',
 'camellia sinensis callus culture extract',
 'magnolia obovata bark extract',
 'theanine',
 'peg/ppg-17/6 copolymer',
 'hydrogenated lecithin',
 'hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer',
 'glycerin',
 'carbomer',
 'cellulose gum',
 'propanediol',
 'xanthan gum',
 '1 2-hexanediol',
 'tromethamine',
 'ethylhexylglycerin',
 'disodium edta',
 'phenoxyethanol',
 'fragrance']

### Si no encuentra el producto, se debe sacar una foto

# Sacar foto (OCR)

In [118]:
def detect_text(path):
    """Detects text in the file."""
    # from google.cloud import vision
    # import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = str((response.text_annotations[0]).description)
    return texts.split(',')
    

In [119]:
def cleaning(detect_text):
    return [''.join(filter(lambda i: i not in "#$%&\'()*+,./:;<=>?@[\\]^_`{|}~\n", item.lower().strip())) for item in detect_text]

In [120]:
path1 = "1.jpg"
path2 = "2.jpg"
path3 = "3.jpg"

In [121]:
detect_text(path3)

['Premium Skin Conditioning Formula\nIngredients: Water (Aqua)',
 ' Caprylic/Capric Triglyceride',
 ' Glycerin',
 '\nPropylene Glycol',
 ' Glyceryl Stearate SE',
 ' Isopropyl Palmitate',
 '\nFREE Stearic Acid',
 ' Glyceryl Stearate',
 ' Cetyl Alcohol',
 ' Stearyl Alcohol\nDimethicone',
 ' Cetearyl Alcohol',
 ' Cetyl Palmitate',
 ' Hydrolyzed\nCollagen',
 ' Phenoxyethanol',
 ' PEG-100 Stearate',
 ' Triethanolamine',
 '\nCeteareth-20',
 ' Shea Butter',
 ' Diazolidinyl Urea',
 ' Fragrance (Parfum)\nTocopheryl Acetate',
 ' Ethylhexylglycerin',
 ' Carbomer',
 ' Panthenol',
 '\nDMDM Hydantoin',
 ' Disodium EDTA. Paraben Free.\nen\nDirections: Apply to a cleansed face',
 ' neck and / or body in the\nSCENTED morning and/ or evening.\nPEAR\nLOT Number on Jar Sur']

In [122]:
text3 = cleaning(detect_text(path3))
len(text3)

26

In [123]:
find_description(text3)

Found 65.38% of ingredients


['A natural moisturizer that’s also in our skin A super common safe effective and cheap molecule used for more than 50 years Not only a simple moisturizer but knows much more keeps the skin lipids between our skin cells in a healthy liquid crystal state protects against irritation helps to restore barrier Effective from as low as 3 with even more benefits at higher concentrations up to 2040 around 10 is a good usabilityeffectiveness sweet spot Highglycerin moisturizers are awesome for treating severely dry skin Glycerin doesn’t sound very glamorous but it is a real oldie but a goodie It’s been used in cosmetics for more than 50 years and it’s a totally natural ingredient that’s also in the outermost layer of our skin called stratum corneum SC It’s a socalled humectant type of moisturiser meaning that it helps our skin to cling onto water A 2005 study has shown that glycerin levels correlate with skin hydration levels so more glycerin means better hydrated skin\nAs newer studies examine

In [112]:
#def clean_ingredient(detect_text):
#    for c in string_punctuation:
#        for i, item in enumerate(detect_text):
#            if c in item:
#                item = item.replace(c,'')
#                detect_text[i] = item
#            return detect_text

In [47]:
#def search_list(test_text):
#    description_list = []
#    for word in text3:
#        if word in data["names"].str.lower():
#            description_list.append(data["description"])
#    return description_list

In [59]:
search_list(test_text)

[]

In [27]:
#Chequeando que el nombre esté en el dataset
data.loc[data['names'].str.contains("glycerin", case=False)]

,names,links,description
111,Capryloyl Glycerin/​Sebacic Acid Copolymer,https://incidecoder.com//ingredients/capryloyl...,The friend of Diheptyl Succinate as the duo is...
256,Diglycerin,https://incidecoder.com//ingredients/diglycerin,The big brother of glycerin Its also a natural...
314,Ethylhexylglycerin,https://incidecoder.com//ingredients/ethylhexy...,If you have spotted ethylhexylglycerin on the ...
337,Glycerin,https://incidecoder.com//ingredients/glycerin,A natural moisturizer that’s also in our skin ...
587,PEG/​PPG/​Polybutylene Glycol-8/​5/​3 Glycerin,https://incidecoder.com//ingredients/peg-ppg-p...,A glycerin derived synthetic oily liquid that ...
662,Polyglycerin-3,https://incidecoder.com//ingredients/polyglyce...,Three glycerin molecules attached together It ...
